## Data Preparation

TODO: add text about data set (summary from data exploration)

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os

#include utils directory
import sys
sys.path.append(os.path.join('..', 'utils'))

from files import *
DATA_PATH = os.path.join('..', 'data')


#### Teams & Coaches

In [10]:
### Teams

teams_df = pd.read_csv(os.path.join(DATA_PATH, DATA_TEAMS))
merged_df = teams_df[['year', 'tmID', 'playoff']]

# Remove unnecessary info
teams_df.drop(columns=['franchID', 'lgID', 'confID', 'divID', 'seeded', 'firstRound', 'semis', 'finals', 'name'], inplace=True)
# Remove nulls
teams_df.drop(columns=["tmORB","tmDRB","tmTRB","opptmORB","opptmDRB","opptmTRB", 'attend', 'arena'], inplace=True)
# Collapse wins & losses into one feature
teams_df['teamWLRatio'] = teams_df['won'] / teams_df['lost']
teams_df.drop(columns=['won', 'lost', 'GP', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'min'], inplace=True)
# Clean up rebounds
teams_df.drop(columns=['o_oreb', 'o_dreb', 'd_dreb', 'd_oreb'], inplace=True)
# Merge
teams_df['year'] = teams_df['year'] + 1
teams_df['playoff_prev'] = teams_df['playoff']
teams_df['rank_prev'] = teams_df['rank']
teams_df.drop(columns=['playoff', 'rank'], inplace=True)

#drop everything
merged_df = pd.merge(merged_df, teams_df, on=['year', 'tmID'], how='left')

### Coaches
coaches_df = pd.read_csv(os.path.join(DATA_PATH, DATA_COACHES))
# Collapse wins & losses into one feature
coaches_df['coachWLRatio'] = (coaches_df['won'] + coaches_df['post_wins']) / (coaches_df['lost'] + coaches_df['post_losses'])
# Remove unnecessary info
coaches_df.drop(columns=['lgID', 'stint', 'won', 'lost', 'post_wins', 'post_losses'], inplace=True)
# Merge
coaches_temp = coaches_df[['coachID', 'year', 'tmID']]
merged_df = pd.merge(merged_df, coaches_temp, on=['year', 'tmID'], how='left')
coaches_df['year'] = coaches_df['year'] + 1
coaches_df.drop(columns=['tmID'], inplace=True)
merged_df = pd.merge(merged_df, coaches_df, on=['year', 'coachID'], how='left')
merged_df['coachWLRatio'] = merged_df['coachWLRatio'].fillna(merged_df['coachWLRatio'].mean())



#### Players and Awards

In [11]:
# Players
players_teams_df = pd.read_csv(os.path.join(DATA_PATH, DATA_PLAYERS_TEAMS))

merging = ['minutes', 'points', 'oRebounds', 'dRebounds', 'assists', 'steals', 'blocks', 'turnovers',
           'PF', 'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade', 'dq']
new_pt_df = pd.DataFrame()

hasUpper = lambda s: any(x.isupper() for x in s)
col = 'minutes'
for col in merging:
    postName = 'Post' + (col if hasUpper(col) else col.capitalize())
    if col == 'dq':
        postName = 'PostDQ'
    new_pt_df[col] = players_teams_df[col] + players_teams_df[postName]

# Make everything a ratio of minutes
for col in new_pt_df:
    if col == 'minutes':
        continue
    new_pt_df[col] = new_pt_df[col] / new_pt_df['minutes']
new_pt_df.drop(columns=['minutes'], inplace=True)

for col in ['playerID', 'year', 'tmID']:
    new_pt_df[col] = players_teams_df[col]

print(new_pt_df.shape)
print(new_pt_df.head())

# get the average stats for players the previous year
for i in range(len(merged_df)):
    cur = new_pt_df[(new_pt_df['tmID'] == merged_df['tmID'][i])
              & (new_pt_df['year'] <= merged_df['year'][i] - 1)].copy()
    cur.drop(columns=['playerID', 'year', 'tmID'], inplace=True)
    for col in cur:
        merged_df[col] = cur[col].mean()
        #pass

'''
players_df = pd.read_csv(os.path.join(DATA_PATH, DATA_PLAYERS))
awards_df = pd.read_csv(os.path.join(DATA_PATH, DATA_AWARDS))

awards_df.drop(columns=['lgID'], inplace=True)

series_post_df = pd.read_csv(os.path.join(DATA_PATH, DATA_SERIES_POST))
teams_post_df = pd.read_csv(os.path.join(DATA_PATH, DATA_TEAMS_POST))
players_df.rename(columns={"bioID": "playerID"}, inplace=True)
# merged_df = pd.merge(merged_df, players_df, on=['playerID'], how='inner')
#merged_df = pd.merge(merged_df, players_teams_df, on=['tmID','year'], how='left')

# merged_df = pd.merge(merged_df, awards_df, on=['playerID', 'year'], how='left')
'''

(1876, 18)
     points  oRebounds  dRebounds   assists    steals    blocks  turnovers   
0  0.405437   0.050827   0.154846  0.062648  0.049645  0.010638   0.100473  \
1  0.390062   0.055901   0.125466  0.074534  0.052174  0.012422   0.114286   
2  0.396051   0.052265   0.117305  0.099884  0.055749  0.013937   0.113821   
3  0.313800   0.037807   0.119093  0.090737  0.058601  0.007561   0.086957   
4  0.391248   0.037323   0.100386  0.077220  0.061776  0.007722   0.102960   

         PF  fgAttempted    fgMade  ftAttempted    ftMade  threeAttempted   
0  0.082742     0.346336  0.134752     0.156028  0.113475        0.089835  \
1  0.090683     0.392547  0.147826     0.144099  0.069565        0.074534   
2  0.101045     0.356562  0.137050     0.123113  0.089431        0.103368   
3  0.092628     0.306238  0.107750     0.094518  0.056711        0.115312   
4  0.110682     0.355212  0.140283     0.093951  0.068211        0.105534   

   threeMade        dq    playerID  year tmID  
0   0.022

'\nplayers_df = pd.read_csv(os.path.join(DATA_PATH, DATA_PLAYERS))\nawards_df = pd.read_csv(os.path.join(DATA_PATH, DATA_AWARDS))\n\nawards_df.drop(columns=[\'lgID\'], inplace=True)\n\nseries_post_df = pd.read_csv(os.path.join(DATA_PATH, DATA_SERIES_POST))\nteams_post_df = pd.read_csv(os.path.join(DATA_PATH, DATA_TEAMS_POST))\nplayers_df.rename(columns={"bioID": "playerID"}, inplace=True)\n# merged_df = pd.merge(merged_df, players_df, on=[\'playerID\'], how=\'inner\')\n#merged_df = pd.merge(merged_df, players_teams_df, on=[\'tmID\',\'year\'], how=\'left\')\n\n# merged_df = pd.merge(merged_df, awards_df, on=[\'playerID\', \'year\'], how=\'left\')\n'

#### Playoffs

In [12]:


'''
# Merge Series_post and Teams twice (for winning and losing teams)
# using "tmIDWinner" and "lgIDWinner" keys for winning teams
winning_teams_df = pd.merge(series_post_df, teams_df, left_on=['year', 'tmIDWinner', 'lgIDWinner'], right_on=[
                            'year', 'tmID', 'lgID'], suffixes=('', '_winning'), how='inner')
# using "tmIDLoser" and "lgIDLoser" keys for losing teams
losing_teams_df = pd.merge(series_post_df, teams_df, left_on=['year', 'tmIDLoser', 'lgIDLoser'], right_on=[
                           'year', 'tmID', 'lgID'], suffixes=('', '_losing'), how='inner')

'''
# Merge Teams_post and Teams using the combination of "year," "tmID," and "lgID" keys
# merged_df = pd.merge(merged_df, teams_post_df, on=['year', 'tmID', 'lgID'], how='inner')





'\n# Merge Series_post and Teams twice (for winning and losing teams)\n# using "tmIDWinner" and "lgIDWinner" keys for winning teams\nwinning_teams_df = pd.merge(series_post_df, teams_df, left_on=[\'year\', \'tmIDWinner\', \'lgIDWinner\'], right_on=[\n                            \'year\', \'tmID\', \'lgID\'], suffixes=(\'\', \'_winning\'), how=\'inner\')\n# using "tmIDLoser" and "lgIDLoser" keys for losing teams\nlosing_teams_df = pd.merge(series_post_df, teams_df, left_on=[\'year\', \'tmIDLoser\', \'lgIDLoser\'], right_on=[\n                           \'year\', \'tmID\', \'lgID\'], suffixes=(\'\', \'_losing\'), how=\'inner\')\n\n'

In [13]:
merged_df['playoff'] = merged_df['playoff'].eq('Y').mul(1)
merged_df = merged_df.select_dtypes(['number']) # Remove later
merged_df.dropna(axis=0, inplace=True)
merged_df.head()

print(merged_df.shape)
merged_df.head()

(140, 46)


,year,playoff,o_fgm,o_fga,o_ftm,o_fta,o_3pm,o_3pa,o_reb,o_asts,...,blocks,turnovers,PF,fgAttempted,fgMade,ftAttempted,ftMade,threeAttempted,threeMade,dq
1,10,1,895.0,2258.0,542.0,725.0,202.0,598.0,1077.0,492.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054
3,2,1,812.0,1903.0,431.0,577.0,131.0,386.0,935.0,551.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054
4,3,1,746.0,1780.0,410.0,528.0,153.0,428.0,948.0,467.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054
5,4,1,770.0,1790.0,490.0,663.0,211.0,527.0,955.0,496.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054
6,5,0,787.0,1881.0,456.0,590.0,187.0,517.0,971.0,499.0,...,0.013457,0.073405,0.118195,0.277315,0.106692,0.090884,0.066904,0.06851,0.021332,0.00054


In [14]:
# Save the result to a new CSV file
merged_df.to_csv(os.path.join(DATA_PATH, DATA_MERGED), index=False)